In [4]:
import datetime
import re
import requests
import json
import warnings

import pandas as pd

RAPID_API_KEY = '67f58679fcmsh1d5d86c70be0778p1320e2jsn414d9154fae0'
NEWS_API_KEY = '7b294a567b694de5b23397387713d7f6' 

keyword = "oil gas"
ticker = "DVN" 

date_formats = ['%Y-%m-%dT%H:%M:%S', '%Y-%m-%d %H:%M:%S', '%d/%m/%Y %H:%M:%S', '%Y-%m-%dT%H:%M:%SZ']
output_file_name= "../data/news/data/news_large1.csv"

# Data Structure for final output
news = {'source': [], 'author': [], 'time': [], 'title': [],
        'description': [], 'content': [], 'url': []}

In [2]:
def addnews(startdate, enddate, news):
    # 1. Contextual Web Search API, 100 Request per day, 50 articles every time
    url1 = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/search/NewsSearchAPI"
    querystring1 = {"q":keyword,"pageNumber":"1","pageSize":"50","autoCorrect":"true","fromPublishedDate":startdate,
                   "toPublishedDate":enddate}
    headers1 = {
        'x-rapidapi-key': RAPID_API_KEY,
        'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com"
        }

    response1 = requests.request("GET", url1, headers=headers1, params=querystring1)

    try: 
        body1 = json.loads(response1.text)
        articles1 = body1["value"]
        n_articles = len(articles1)

        for i, article in enumerate(articles1):
            news['source'].append(article['provider']['name'])
            news['author'].append("NotProvided")
            for date_format in date_formats:
                try:
                    news['time'].append(datetime.datetime.strptime(article['datePublished'], date_format))
                    break
                except:
                    continue
            news['title'].append(article['title'])
            news['description'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', article['description']).strip())
            news['content'].append(re.sub(r'(\r|\n|<.*?>|… \[\+[0-9]+ chars])+', ' ', article['body']).strip())
            news['url'].append(article['url'])
    except:
        warnings.warn("1. Fail to get news from Contextual Web Search API!")


    # 2. Newscatcher API, 25 Request per hour, only five articles every time
    # doc: https://newscatcherapi.com/news-api
    url21 = "https://newscatcher.p.rapidapi.com/v1/stocks"
    querystring21 = {"ticker":ticker,"from":startdate,"to":enddate,"lang":"en","stock":"NASDAQ","media":"True","sort_by":"relevancy"}

    headers21 = {
        'x-rapidapi-key': RAPID_API_KEY,
        'x-rapidapi-host': "newscatcher.p.rapidapi.com"
        }

    response21 = requests.request("GET", url21, headers=headers21, params=querystring21)

    try: 
        body21 = json.loads(response21.text)
        articles21 = body21["articles"]
        n_articles21 = len(articles21)

        for i, article in enumerate(articles21):
            news['source'].append("NotProvided")
            news['author'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', str(article['author'])).strip())
            for date_format in date_formats:
                try:
                    news['time'].append(datetime.datetime.strptime(article['published_date'], date_format))
                    break
                except:
                    continue
            news['title'].append(article['title'])
            news['description'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', article['summary']).strip())
            news['content'].append(' ')
            news['url'].append(article['link'])
    except Exception as e:
        print(e)
        warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")


    url22 = "https://newscatcher.p.rapidapi.com/v1/search"
    querystring22 = {"q":keyword,"from":startdate,"to":enddate,"lang":"en","media":"True","sort_by":"relevancy"}

    headers22 = {
        'x-rapidapi-key': RAPID_API_KEY,
        'x-rapidapi-host': "newscatcher.p.rapidapi.com"
        }

    response22 = requests.request("GET", url22, headers=headers22, params=querystring22)

    try: 
        body22 = json.loads(response22.text)
        articles22 = body22["articles"]
        n_articles22 = len(articles22)

        for i, article in enumerate(articles22):
            news['source'].append("NotProvided")
            news['author'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', str(article['author'])).strip())
            for date_format in date_formats:
                try:
                    news['time'].append(datetime.datetime.strptime(article['published_date'], date_format))
                    break
                except:
                    continue
            news['title'].append(article['title'])
            news['description'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', article['summary']).strip())
            news['content'].append(' ')
            news['url'].append(article['link'])
    except Exception as e:
        print(e)
        warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


    # 3. NewsAPI search, 100 requests per day
    url3 = ('https://newsapi.org/v2/everything?'
           'q='+keyword+'&'
           'from='+startdate+'&'
           'to='+enddate+'&'
           'sortBy=relevancy&'
           'pageSize=100&'
           'apiKey='+NEWS_API_KEY)

    try: 
        articles3 = requests.get(url3).json()['articles']

        for i, article in enumerate(articles3):
            news['source'].append(article['source']['name'])
            news['author'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', str(article['author'])).strip())
            for date_format in date_formats:
                try:
                    news['time'].append(datetime.datetime.strptime(article['publishedAt'], date_format))
                    break
                except:
                    continue
            news['title'].append(article['title'])
            news['description'].append(re.sub(r'(\r|\n|<.*?>|…)+', ' ', article['description']).strip())
            news['content'].append(re.sub(r'(\r|\n|<.*?>|… \[\+[0-9]+ chars])+', ' ', article['content']).strip())
            news['url'].append(article['url'])
    except:
        warnings.warn("3. Fail to get news from NewsAPI Search!")

In [5]:
startdate = "2016-03-31T00:00:00"
enddate = "2016-05-31T23:59:59"
addnews(startdate,enddate, news)

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")
<ipython-input-2-649c75e1d83d>:129: UserWarning: 3. Fail to get news from NewsAPI Search!
  warnings.warn("3. Fail to get news from NewsAPI Search!")


In [6]:
startdate = "2016-05-31T00:00:00"
enddate = "2016-07-31T23:59:59"
addnews(startdate,enddate, news)

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")
<ipython-input-2-649c75e1d83d>:129: UserWarning: 3. Fail to get news from NewsAPI Search!
  warnings.warn("3. Fail to get news from NewsAPI Search!")


In [7]:
startdate = "2016-07-31T00:00:00"
enddate = "2016-09-31T23:59:59"
addnews(startdate,enddate, news)

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")
<ipython-input-2-649c75e1d83d>:129: UserWarning: 3. Fail to get news from NewsAPI Search!
  warnings.warn("3. Fail to get news from NewsAPI Search!")


In [8]:
startdate = "2016-09-30T00:00:00"
enddate = "2016-10-31T23:59:59"
addnews(startdate,enddate, news)

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")
<ipython-input-2-649c75e1d83d>:129: UserWarning: 3. Fail to get news from NewsAPI Search!
  warnings.warn("3. Fail to get news from NewsAPI Search!")


In [9]:
len(news['source'])

61

In [10]:
startdate = "2016-10-31T00:00:00"
enddate = "2016-11-31T23:59:59"
addnews(startdate,enddate, news)

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")
<ipython-input-2-649c75e1d83d>:129: UserWarning: 3. Fail to get news from NewsAPI Search!
  warnings.warn("3. Fail to get news from NewsAPI Search!")


In [11]:
len(news['source'])

111

In [12]:
startdate = "2016-11-31T00:00:00"
enddate = "2016-12-15T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


111

In [13]:
startdate = "2016-11-31T00:00:00"
enddate = "2017-02-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


213

In [14]:
startdate = "2017-02-31T00:00:00"
enddate = "2017-04-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


315

In [15]:
startdate = "2017-04-31T00:00:00"
enddate = "2017-06-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


417

In [16]:
startdate = "2017-06-31T00:00:00"
enddate = "2017-09-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


519

In [17]:
startdate = "2017-09-31T00:00:00"
enddate = "2017-11-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


621

In [18]:
startdate = "2017-11-31T00:00:00"
enddate = "2018-02-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


723

In [19]:
startdate = "2018-02-31T00:00:00"
enddate = "2018-04-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


825

In [20]:
startdate = "2018-04-31T00:00:00"
enddate = "2018-06-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


927

In [21]:
startdate = "2018-06-31T00:00:00"
enddate = "2018-09-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1029

In [22]:
startdate = "2018-09-31T00:00:00"
enddate = "2018-11-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1131

In [23]:
startdate = "2018-11-31T00:00:00"
enddate = "2019-02-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1233

In [24]:
startdate = "2019-02-31T00:00:00"
enddate = "2019-04-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1335

In [25]:
startdate = "2019-04-31T00:00:00"
enddate = "2019-06-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1437

In [26]:
startdate = "2019-06-31T00:00:00"
enddate = "2019-09-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1539

In [27]:
startdate = "2019-09-31T00:00:00"
enddate = "2019-11-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1641

In [28]:
startdate = "2019-11-31T00:00:00"
enddate = "2020-02-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1743

In [29]:
startdate = "2020-02-31T00:00:00"
enddate = "2020-04-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1845

In [30]:
startdate = "2020-04-31T00:00:00"
enddate = "2020-06-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


1947

In [31]:
startdate = "2020-06-31T00:00:00"
enddate = "2020-09-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


2049

In [32]:
startdate = "2020-09-31T00:00:00"
enddate = "2020-11-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


2151

In [33]:
startdate = "2020-11-31T00:00:00"
enddate = "2021-02-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


2253

In [34]:
startdate = "2021-02-31T00:00:00"
enddate = "2021-03-31T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")


2253

In [35]:
startdate = "2021-03-31T00:00:00"
enddate = "2021-04-16T23:59:59"
addnews(startdate,enddate, news)
len(news['source'])

'articles'
<ipython-input-2-649c75e1d83d>:67: UserWarning: 21. Fail to get news from Newscatcher API Stock Search!
  warnings.warn("21. Fail to get news from Newscatcher API Stock Search!")
'articles'
<ipython-input-2-649c75e1d83d>:100: UserWarning: 22. Fail to get news from Newscatcher API Keyword Search!
  warnings.warn("22. Fail to get news from Newscatcher API Keyword Search!")
<ipython-input-2-649c75e1d83d>:129: UserWarning: 3. Fail to get news from NewsAPI Search!
  warnings.warn("3. Fail to get news from NewsAPI Search!")


2303

In [36]:
df_news = pd.DataFrame(news)
df_news.to_csv("../data/news_large.csv", index=False)